In [2]:
from qiskit import QuantumCircuit, QuantumRegister
import numpy as np
import qiskit.quantum_info as qi
from qiskit import transpile 
from qiskit.providers.fake_provider import FakeWashingtonV2

def create_shift_matrix(N):
    # Create an n x n identity matrix using NumPy
    op = np.identity(N)
    # Swap the first and last rows
    op = np.roll(op,1,axis=0)
    return op

# circuit from matrix experiment
nq = 2
n = 2**nq
P = create_shift_matrix(n)

ccn_gate = qi.Operator(P)
ccn_qc = QuantumCircuit(nq)
ccn_qc.unitary(ccn_gate, range(nq), label="Shift")

## transpile circuit from matrix

backend = FakeWashingtonV2()

transpiled_qc = transpile(ccn_qc,
                          backend=backend,
                          optimization_level=2)
print("Circuit depth: " + str(transpiled_qc.depth()))
# transpiled_qc.draw()

# ccn_qc.draw("mpl")
# print(P)

Circuit depth: 3


In [2]:
def shift(N):
    qc = QuantumCircuit(N)
    qreg = QuantumRegister(N, 'q')

    for i in reversed(range(1, N)):
        qc.mct(qreg[:i], qreg[i])
    qc.x(qreg[0])

    return qc

# implement shift operator directly
# c3 = QuantumCircuit(4)
# c3.mct([0,1,2], [3])
# c3.mct([0,1], [2])
# c3.mct([0], [1])
# c3.x(0)
# c3.draw(fold=-1)
# print(qi.Operator(c3).data.real)
nq = 2
c3 = shift(nq)

transpiled_qc = transpile(c3,
                          backend=backend,
                          optimization_level=3)
print("Circuit depth: " + str(transpiled_qc.depth()))
# transpiled_qc.draw()

Circuit depth: 2


In [8]:
# experiment from O(n) incrementer circuit
def create_incrementer(N):
    
    if N == 1:
        qc = QuantumCircuit(1)
        qc.x(0)
        return qc
    elif N == 2:
        qc = QuantumCircuit(2)
        qc.cx(0,1)
        qc.x(0)
        return qc
    
    nq_total = N + (N - 2)

    qc = QuantumCircuit(nq_total)

    # add Toffoli gates
    for i in range(0,N-2):
        qc.ccx(i*2,i*2 + 1, i*2 + 2)
    qc.cx(nq_total - 2, nq_total - 1)
    qc.barrier()

    # add CX gates
    for i in range(N-3,-1,-1):
        qc.ccx(i*2,i*2 + 1, i*2 + 2)
        qc.cx(i*2, i*2 + 1)

    qc.x(0)

    return qc

nq = 4
inc = create_incrementer(nq)
print(inc)

# print(qi.Operator(inc).data.real)
# inc.draw(fold=-1)

## qiskit transpilation
transpiled_qc = transpile(inc,
                          backend=backend,
                          optimization_level=3)
print("Circuit depth: " + str(transpiled_qc.depth()))
# transpiled_qc.draw()



                     ░                     ┌───┐
q_0: ──■─────────────░─────────────■────■──┤ X ├
       │             ░             │  ┌─┴─┐└───┘
q_1: ──■─────────────░─────────────■──┤ X ├─────
     ┌─┴─┐           ░           ┌─┴─┐└───┘     
q_2: ┤ X ├──■────────░───■────■──┤ X ├──────────
     └───┘  │        ░   │  ┌─┴─┐└───┘          
q_3: ───────■────────░───■──┤ X ├───────────────
          ┌─┴─┐      ░ ┌─┴─┐└───┘               
q_4: ─────┤ X ├──■───░─┤ X ├────────────────────
          └───┘┌─┴─┐ ░ └───┘                    
q_5: ──────────┤ X ├─░──────────────────────────
               └───┘ ░                          
Circuit depth: 83


In [32]:
# organize incrementer in intuitive order
def qubit_list(qreg, inc_reg):

    q_list = [qreg[0]]
    for i in range(len(qreg)-2):
        q_list.append(qreg[i+1])
        q_list.append(inc_reg[i])
    q_list.append(qreg[-1])

    return q_list

nq = 4
qreg = QuantumRegister(nq, 'qreg')
inc_reg = QuantumRegister(nq - 2, 'inc_ancilla')
inc = QuantumCircuit(qreg, inc_reg)

# implement linear shift operator
inc.compose(create_incrementer(nq), qubits=qubit_list(qreg, inc_reg), inplace=True)
print(inc)

                               ░                     ┌───┐
       qreg_0: ──■─────────────░─────────────■────■──┤ X ├
                 │             ░             │  ┌─┴─┐└───┘
       qreg_1: ──■─────────────░─────────────■──┤ X ├─────
                 │             ░      ┌───┐  │  └───┘     
       qreg_2: ──┼────■────────░───■──┤ X ├──┼────────────
                 │    │  ┌───┐ ░   │  └─┬─┘  │            
       qreg_3: ──┼────┼──┤ X ├─░───┼────┼────┼────────────
               ┌─┴─┐  │  └─┬─┘ ░   │    │  ┌─┴─┐          
inc_ancilla_0: ┤ X ├──■────┼───░───■────■──┤ X ├──────────
               └───┘┌─┴─┐  │   ░ ┌─┴─┐     └───┘          
inc_ancilla_1: ─────┤ X ├──■───░─┤ X ├────────────────────
                    └───┘      ░ └───┘                    


In [35]:
from qiskit.quantum_info import Statevector

# verifiy optimized circuit evolves state accordingly
initial_state = Statevector.from_label('001111')
print(initial_state.data.real)
# Evolve the initial state using the circuit
final_state = initial_state.evolve(inc)

print(final_state.data.real)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [75]:
def quantum_subtracter(N):
    nq_total = 2*N
    qc = QuantumCircuit(nq_total)

    for i in range(0, N-1):
        qc.cx(2*i, 2*i+1)
        qc.cx(2*i+2, 2*i)
        qc.ccx(2*i, 2*i + 1, 2*i + 2)
    
    qc.cx(nq_total-2, nq_total-1)

    for i in range(N-2,-1,-1):
        qc.ccx(2*i, 2*i+1, 2*i + 2)
        qc.cx(2*i+2, 2*i)
        qc.cx(2*i+2, 2*i+1)
        # qc.barrier()

    return qc

def general_quantum_incrementer(N):
    nq_total = 2*N
    qc = QuantumCircuit(nq_total)

    target_bits = [ 2*i + 1 for i in range(N)]
    qc.mcx([0], target_bits)
    # qc.barrier()

    for i in range(1, N):
        qc.x(2*i)
    qc.x(2*N - 1)
    # qc.barrier()

    # subtractor
    sub = quantum_subtracter(N)
    qc.compose(sub, inplace=True)

    for i in range(1, N):
        qc.x(2*i)

    # qc.barrier()

    # subtractor
    qc.compose(sub, inplace=True)

    qc.mcx([0], target_bits)

    return qc

In [89]:
nq = 10
# ginc = quantum_subtracter(nq)
ginc = general_quantum_incrementer(nq)

# print(qi.Operator(inc).data.real)
# ginc.draw(fold=-1)

transpiled_qc = transpile(ginc,
                          backend=backend,
                          optimization_level=3)
print("Circuit depth: " + str(transpiled_qc.depth()))

# from qiskit.quantum_info import Statevector

# # verifiy optimized circuit evolves state accordingly
# initial_state = Statevector.from_label('00')
# print(initial_state.data.real)
# # Evolve the initial state using the circuit
# final_state = initial_state.evolve(ginc)

# print(final_state.data.real)


Circuit depth: 1032


In [ ]:
# BQSKit tooling
import bqskit
from bqskit.ext import qiskit_to_bqskit, bqskit_to_qiskit
from bqskit.compiler.compile import build_workflow


c3_bqskit = qiskit_to_bqskit(c3)
default_opt1_workflow = build_workflow(c3_bqskit, optimization_level=1)
print(default_opt1_workflow)

In [ ]:
from bqskit.compiler import Workflow
from bqskit.passes import QuickPartitioner, ForEachBlockPass, ScanningGateRemovalPass, UnfoldPass

basic_gate_deletion_workflow = Workflow([
    QuickPartitioner(3),  # Partition into 3-qubit blocks
    ForEachBlockPass(ScanningGateRemovalPass()),  # Apply gate deletion to each block (in parallel)
    UnfoldPass(),  # Unfold the blocks back into the original circuit
])

In [ ]:
from bqskit.compiler import Compiler

with Compiler() as compiler:
    opt_circuit = compiler.compile(qiskit_to_bqskit(c3), workflow=basic_gate_deletion_workflow)

print(opt_circuit.gate_counts)
c3_opt = bqskit_to_qiskit(opt_circuit)
print("Circuit depth: " + str(c3_opt.depth()))
c3_opt.draw()